In [ ]:
import pandas as pd
from datasets import concatenate_datasets, load_dataset

Reading the raw dataset.

In [ ]:
ds = load_dataset(
    "cornell-movie-review-data/rotten_tomatoes",
    revision="aa13bc287fa6fcab6daf52f0dfb9994269ffea28"
)
df = concatenate_datasets(ds.values()).to_pandas()
df

Dropping useless columns.

In [ ]:
df = df.drop(columns=["label"])

Dropping rows with missing data.

In [ ]:
df = df.dropna()

Renaming columns.

In [ ]:
df = df.rename(columns={"text": "body"})

Adding the politicalness label column.

In [ ]:
df["politicalness"] = pd.Categorical(["non-political"] * len(df))

Inspecting body length.

In [ ]:
df["body_length"] = df["body"].str.len()
df["body_word_count"] = df["body"].str.split().str.len()
df = df.sort_values(by="body_length")
df.head()

After inspection, rows with bodies shorter than 3 words seem to contain no useful value. Removing them.

In [ ]:
body_word_count_lower_bound = 3
df = df[df["body_word_count"] >= body_word_count_lower_bound]
# Ensuring to include the last (longest) item.
downsampled = pd.concat([df["body_length"].iloc[::100], df["body_length"].tail(1)]).drop_duplicates()
downsampled.plot.bar().xaxis.set_ticks([]);

In [ ]:
df["body_length"].mean()

In [ ]:
df.to_parquet("../preprocessed/rotten_tomatoes.parquet")